In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


In [ ]:
df = pd.read_csv(r"/Users/pramukhvenkateshkoushik/Desktop/cleaned.csv" )
df

In [ ]:
class RestaurantRecommendationSystem:
    def __init__(self, restaurants_df):
        self.restaurants_df = restaurants_df
        self.prepare_data()
        
    def prepare_data(self):
        self.restaurants_df['menu_list'] = self.restaurants_df['menu'].apply(eval)

        self.create_menu_matrix()
   
        self.create_cuisine_matrix()
        
    def create_menu_matrix(self):
        all_menu_items = set()
        for menu in self.restaurants_df['menu_list']:
            all_menu_items.update(menu)
    
        menu_matrix = pd.DataFrame(0, index=self.restaurants_df.index,columns=list(all_menu_items))
        
        for idx, menu in enumerate(self.restaurants_df['menu_list']):
            menu_matrix.iloc[idx][menu] = 1   
        self.menu_matrix = menu_matrix
        #print(menu_matrix)
        
    def create_cuisine_matrix(self):
        self.cuisine_matrix = pd.get_dummies(self.restaurants_df['cuisine'])
        #print(self.cuisine_matrix)
        
    def get_menu_based_recommendations(self, restaurant_id, n_recommendations=5):
        menu_similarity = cosine_similarity(self.menu_matrix)
        restaurant_idx = self.restaurants_df[self.restaurants_df['restaurant_id'] == restaurant_id].index[0]
        similarity_scores = list(enumerate(menu_similarity[restaurant_idx]))
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        recommendations = []
        for idx, score in similarity_scores[1:n_recommendations+1]:
            rest_id = self.restaurants_df.iloc[idx]['restaurant_id']
            rest_name = self.restaurants_df.iloc[idx]['name']
            recommendations.append({
                'restaurant_id': rest_id,
                'name': rest_name,
                'similarity_score': score,
                'cuisine': self.restaurants_df.iloc[idx]['cuisine']
            })
            
        return recommendations
    
    def get_cuisine_based_recommendations(self, cuisine_type, n_recommendations=5):
        cuisine_restaurants = self.restaurants_df[
            self.restaurants_df['cuisine'] == cuisine_type
        ].sort_values('stars', ascending=False)
        
        return cuisine_restaurants.head(n_recommendations)[
            ['restaurant_id', 'name', 'stars', 'review_count']
        ].to_dict('records')
    
    def perform_menu_analysis(self, min_support=0.1):
        te = TransactionEncoder()
        te_ary = te.fit(self.restaurants_df['menu_list']).transform(self.restaurants_df['menu_list'])
        df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

        frequent_itemsets = apriori(
            df_encoded,
            min_support=min_support,
            use_colnames=True
        )
        #print( frequent_itemsets)
        rules = association_rules(
            frequent_itemsets,
            None, 
            metric="confidence",
            min_threshold=0.5
        )
        return frequent_itemsets, rules
    
    def get_price_based_recommendations(self, price_range, cuisine_type=None, n_recommendations=5):
        mask = self.restaurants_df['PriceRange'] == price_range
        if cuisine_type:
            mask &= self.restaurants_df['cuisine'] == cuisine_type

        recommendations = self.restaurants_df[mask].sort_values(
            ['stars', 'review_count'], 
            ascending=[False, False]
        ).head(n_recommendations)
        
        return recommendations[
            ['restaurant_id', 'name', 'cuisine', 'stars', 'review_count', 'PriceRange']
        ].to_dict('records')


In [ ]:
rec_system = RestaurantRecommendationSystem(df)
rest_id= int(input("Enter the restaurant id"))
print(f"\nMenu-based recommendations for restaurant_id {rest_id}:")
menu_recommendations = rec_system.get_menu_based_recommendations(rest_id)
for rec in menu_recommendations:
    print(f"Restaurant: {rec['name']}, Similarity: {rec['similarity_score']:.2f}")

cuis = input("Enter the cuisine: ")
print(f"\nTop {cuis} restaurants:")
cuisine_recommendations = rec_system.get_cuisine_based_recommendations(cuis)
for rec in cuisine_recommendations:
    print(f"Restaurant: {rec['name']}, Stars: {rec['stars']}")

print("\nMenu item associations:")
frequent_itemsets, rules = rec_system.perform_menu_analysis(min_support=0.01)
print("\nTop association rules:")
print(rules.sort_values('confidence', ascending=False).head())

print("\nRecommendations for price range 2:")
price_recommendations = rec_system.get_price_based_recommendations(
    price_range=2, 
    cuisine_type='American'
)
for rec in price_recommendations:
    print(f"Restaurant: {rec['name']}, Stars: {rec['stars']}, Price Range: {rec['PriceRange']}")